In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:6 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 252 kB in 3s (93.9 kB/s)
Reading package li

In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"
import findspark
findspark.init()

In [4]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-07-16 01:19:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.5’

postgresql-42.2.16. 100%[===================>] 979.38K  1.15MB/s    in 0.8s    

2022-07-16 01:19:14 (1.15 MB/s) - ‘postgresql-42.2.16.jar.5’ saved [1002883/1002883]



In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [6]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Camera_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("amazon_reviews_us_Camera_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|    2975964|R1NBG94582SJE2|B00I01JQJM|     860486164|GoPro Rechargeabl...|          Camera|          5|            0|          0|   N|                Y|          Five Stars|                  ok| 2015-08-31|
|         US|   23526356|R273DCA6Y0H9V7|B00TCO0ZAA|     292641483|Professional 58mm...|          Camera|          5|    

In [7]:
df.dtypes

[('marketplace', 'string'),
 ('customer_id', 'int'),
 ('review_id', 'string'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('product_title', 'string'),
 ('product_category', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string'),
 ('review_headline', 'string'),
 ('review_body', 'string'),
 ('review_date', 'string')]

In [8]:
# Change the datatypes of following columns:
#  customer_id, product_parent, star_rating, helpful_votes, total_votes
from pyspark.sql.types import IntegerType

In [9]:
from pyspark.sql.functions import to_date
# Review DataFrame
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R1NBG94582SJE2|    2975964|B00I01JQJM|     860486164| 2015-08-31|
|R273DCA6Y0H9V7|   23526356|B00TCO0ZAA|     292641483| 2015-08-31|
| RQVOXO7WUOFK6|   52764145|B00B7733E0|      75825744| 2015-08-31|
|R1KWKSF21PO6HO|   47348933|B006ZN4U34|     789352955| 2015-08-31|
|R38H3UO1J190GI|   33680700|B00HUEBGMU|      19067902| 2015-08-31|
|R3NPIFKLR19NQA|   30301059|B008MW6Y12|     597683407| 2015-08-31|
|R3MBE6UCH3435E|   28282645|B00TE8XKIS|      35563334| 2015-08-31|
|R2E7A4FF0PVY5Q|     502818|B00ZKDUFBQ|     555190742| 2015-08-31|
| R3R8JDQ2BF4NM|    1481233|B010BZ7S2Q|     129544315| 2015-08-31|
|R1YND4BS823GN5|   27885926|B00HRXSSRA|     708418657| 2015-08-31|
|R2TZNSA18V7YF6|    3183883|B005C95NM4|     246957815| 2015-08-31|
|R22ZVRDPPXIDNL|   23208852|B00LBIBYBE|     746593019| 2015-08

In [10]:
review_id_df.dtypes

[('review_id', 'string'),
 ('customer_id', 'int'),
 ('product_id', 'string'),
 ('product_parent', 'int'),
 ('review_date', 'date')]

In [11]:
products_df = df.select(["product_id", "product_title"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00I01JQJM|GoPro Rechargeabl...|
|B00TCO0ZAA|Professional 58mm...|
|B00B7733E0|Spy Tec Z12 Motio...|
|B006ZN4U34|Celestron UpClose...|
|B00HUEBGMU|Vidpro XM-L Wired...|
|B008MW6Y12|NIX 8 inch Hi-Res...|
|B00TE8XKIS|Polaroid ZIP Mobi...|
|B00ZKDUFBQ|GeekPro 2.0-Inch ...|
|B010BZ7S2Q|Sony HDR-AZ1VR Ac...|
|B00HRXSSRA|ChiliPower DMW-BL...|
|B005C95NM4|Zeikos Deluxe Fla...|
|B00LBIBYBE|  GoPro Hero Filters|
|B00X3HIM2U|Neewer Meike MK-X...|
|B00KDVQF8W|LB Photography Ba...|
|B00GUZEZL4|Waterproof Camera...|
|B00FB1TBKS|Ecolink Z-Wave PI...|
|B00GVMLPT6|FotoTech Male to ...|
|B00L8827BI|Nikon D3200 Ultim...|
|B00SIM78R0|D-Link Wireless D...|
|B00EDCZKJ2|ZINK Phone Photo ...|
+----------+--------------------+
only showing top 20 rows



In [12]:
products_df.count()

1801974

In [13]:
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|B00L9ZF5BY|Flylinktech Actio...|
|B00017LSF8|B+W Clear UV Haze...|
|B00CGTN2N0|Queen Spy Mini U8...|
|B002POMZWU|Usb cable for kod...|
|B00FEY2BCW|Camera Case Eveca...|
|B0131XNIDS|*5-pack* microSue...|
|B004OUMTT2|Celestron PowerSe...|
|B004HY3XOA|58MM UV Lens Filt...|
|B00LBFKLRU|(Create a generic...|
|B007QXZ6D0|Vello Cinch Strap...|
|B00PG36JTW|Tamrac TEK Fitted...|
|B003JNJDGM|TTL Off Camera Sh...|
|B00ZYATKCA|Olympus Air Mirro...|
|B00S2X48IW|Qkoo Selfie Handh...|
|B00VQJASPG|    Canon SL1 Bundle|
|B00554PCDG|Pixel Flash Hot S...|
|B00006JPWW|Panasonic AY-DVMC...|
|B00B9Z23NM|Nikon 8250 ACULON...|
|B00GOJ7JGY|Fotasy HET63 Lens...|
|B00IUU22WG|White Photography...|
+----------+--------------------+
only showing top 20 rows



In [14]:
products_df.dtypes

[('product_id', 'string'), ('product_title', 'string')]

In [15]:
customers_df = df.groupby("customer_id").agg({"customer_id": "count"}).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   52695798|             1|
|   48363612|             5|
|   46909180|             1|
|   45595220|             1|
|   50372387|             2|
|    9731896|             1|
|   24540309|             1|
|    2019000|             1|
|   50798385|             5|
|   37669883|             1|
|   19718301|             1|
|   45616772|             1|
|    2167730|             1|
|   47027968|             7|
|    5459822|             1|
|   52484883|             1|
|   12425248|             1|
|   15460750|             1|
|   46944960|             1|
|   37502310|             2|
+-----------+--------------+
only showing top 20 rows



In [16]:
customers_df.dtypes

[('customer_id', 'int'), ('customer_count', 'bigint')]

In [17]:
customers_df_2 = df.groupby("customer_id").agg({"customer_id": "count"})
customers_df_2.show()

+-----------+------------------+
|customer_id|count(customer_id)|
+-----------+------------------+
|   52695798|                 1|
|   48363612|                 5|
|   46909180|                 1|
|   45595220|                 1|
|   50372387|                 2|
|    9731896|                 1|
|   24540309|                 1|
|    2019000|                 1|
|   50798385|                 5|
|   37669883|                 1|
|   19718301|                 1|
|   45616772|                 1|
|    2167730|                 1|
|   47027968|                 7|
|    5459822|                 1|
|   52484883|                 1|
|   12425248|                 1|
|   15460750|                 1|
|   46944960|                 1|
|   37502310|                 2|
+-----------+------------------+
only showing top 20 rows



In [18]:
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", 'verified_purchase'])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1NBG94582SJE2|          5|            0|          0|   N|                Y|
|R273DCA6Y0H9V7|          5|            0|          0|   N|                Y|
| RQVOXO7WUOFK6|          2|            1|          1|   N|                Y|
|R1KWKSF21PO6HO|          5|            0|          0|   N|                Y|
|R38H3UO1J190GI|          5|            1|          1|   N|                Y|
|R3NPIFKLR19NQA|          3|            0|          0|   N|                Y|
|R3MBE6UCH3435E|          3|            8|          8|   N|                N|
|R2E7A4FF0PVY5Q|          5|            0|          1|   N|                Y|
| R3R8JDQ2BF4NM|          5|            0|          2|   N|                Y|
|R1YND4BS823GN5|          1|            0|          0|   N|     

In [19]:
vine_df.dtypes

[('review_id', 'string'),
 ('star_rating', 'int'),
 ('helpful_votes', 'int'),
 ('total_votes', 'int'),
 ('vine', 'string'),
 ('verified_purchase', 'string')]

In [20]:
mode = "append"
jdbc_url="jdbc:postgresql://endpoint:5432/database"
config = {"user":"postgres",
          "password": password,
          "driver":"org.postgresql.Driver"}

In [21]:
short_review = review_id_df.limit(50)

In [22]:
# Write review_id_df to the table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [23]:
# Write products_df to the table in RDS
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [24]:
# Write customers_df to the table in RDS
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [25]:
# Write vine_df to the table in RDS
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)